<h1 align="center"><font size="6"> Les méthodes de descente de gradient </font> (deuxième partie)</h1>
<hr> 

<h1>Table des matières</h1>

<div class="alert alert-block alert-info" style="margin-top : 20px">
      <ul>
          <li><a href="#prelim">Préliminaires</a></li>
          <li><a href="#Newton">La méthode de Newton</a></li>
          <li><a href="#BFGS">Une méthode de quasi-Newton (BFGS)</a></li>
      </ul>
</div>
<br>
<h>

<a id='prelim'></a>
<h2>Préliminaires</h2>
<hr>

On commence par importer les bibliothèques neecéssaires (_numpy_ et _matplotlib.pyplot_).

On définit aussi deux functions pour la visualisation de: *1/* les lignes de niveaux de la fonction ojectif et *2/* le champ de gradients (pour les fonctions objectifs dépendant de deux variables). 

Il y a aussi un exemple de graphique pour observer la vitesse de convergence des méthodes d'optimisation.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact
from mpl_toolkits.mplot3d import Axes3D

def draw_vector_field(F, xmin, xmax, ymin, ymax, N=15):
    X = np.linspace(xmin, xmax, N)  # x coordinates of the grid points
    Y = np.linspace(ymin, ymax, N)  # y coordinates of the grid points
    U, V = F(*np.meshgrid(X, Y))  # vector field
    M = np.hypot(U, V)  # compute the norm of (U,V)
    M[M == 0] = 1  # avoid division by 0
    U /= M  # normalize the u componant
    V /= M  # normalize the v componant
    return plt.quiver(X, Y, U, V, angles='xy')

def level_lines(f, xmin, xmax, ymin, ymax, levels, N=500):
    x = np.linspace(xmin, xmax, N)
    y = np.linspace(ymin, ymax, N)
    z = f(*np.meshgrid(x, y))
    level_l = plt.contour(x, y, z, levels=levels)
    #plt.clabel(level_l, levels, fmt='%.1f') 

f = lambda x, y : np.cosh(x)+ np.sin(x + y)**2
df = lambda x, y : np.array([np.sinh(x) + 2*np.cos(x + y)*np.sin(x + y), 2*np.cos(x + y)*np.sin(x + y)])
%matplotlib inline
level_lines(f, -1.1, 1.1, -1.1, 1.1, np.linspace(1, 3, 10))
draw_vector_field(df, -1.1, 1.1, -1.1, 1.1, 10)
plt.axis('equal')
plt.show()

# plot of the values of f along the iterations.
N = 10
F = 2**(-np.linspace(0,N,N+1))
plt.figure()
plt.semilogy(range(N + 1), F, '.', linestyle='dashed')

<a id='Newton'></a>
<h2>La méthode de Newton</h2>
<hr>

On suppose dans ce TP que $f:\mathbb{R}^N\to\mathbb{R}$ est de classe $C^2$ au moins.

La méthode de Newton (ou de Newton-Raphson) est une méthode de descente itérative dans laquelle la direction de descente à l'étape $k$ est choisie de manière à minimiser le développement limité au second ordre de $f$ au point $x^k$, c'est-à-dire
$$
\tag{4}
m_k(d):=f(x^k) + d\cdot \nabla f(x^k) + \dfrac12 d^T D^2 f(x^k) d.
$$
Si la matrice (symétrique) $D^2 f(x^k)$ est définie  positive le minimiseur de $m^k$ existe et est unique. On note $H^k$ l'inverse de $D^2 f(x^k)$, $g^k:=\nabla f(x^k)$ et $d^k$ le minimiseur de (4).

<hr>

***Question 15.*** Exprimez $d^k$ en fonction de $H^k$ et $g^k$.

<u>Réponse</u> : Pour trouver $d^k$ on cherche 

$\dfrac{\partial m_k}{\partial d}(d^k)=0$

Or, $\dfrac{\partial m_k}{\partial d}(d)=g^k + D^k d$

Donc $d^k = -H^k g^k$

----------------------------------------------------------------------------------------------------------------

Soit $\Lambda>0$. On pose $f_\Lambda(x,y):=(1-x)^2 + \Lambda\,(y-x^2)^2$, pour $(x,y)\in\mathbb{R}^2$.  

__Question 16.__ Calculez $\nabla f_\Lambda(x,y)$. Trouves le(s) minimiseur(s) de $f_\Lambda$. Tracez quelques lignes de niveau de $f_\Lambda$ ainsi que le champ vectoriel renormalisé $(1/|\nabla f_\Lambda|)\nabla f_\Lambda$ pour $\Lambda=100$. Calculez $D^2 f(x,y)$ et son inverse $H_\Lambda(x,y)$.

<u>Réponse</u> :

$$\nabla f_{\Lambda}(x,y) = 2\binom{-((1-x)+2x\Lambda(y-x^2)}{\Lambda(y-x^2)}$$
$$D^2f_{\Lambda}(x,y) = 2\binom{1-2\Lambda(y-x^2) + 4x^2 \quad -2\Lambda x}{-2\Lambda x \quad \quad \quad \quad \quad \quad \Lambda}$$

In [ ]:
## Solution 
Lambda = 100
f = lambda x,y : ( x - 1)**2 + Lambda*(y - x**2)**2
df = lambda x,y : np.array([2*(x-1)+4*Lambda*x*(x**2-y),2*Lambda*(y-x**2)])
ddf = lambda x,y : np.array([[2+4*Lambda*(3*x**2-y), -4*Lambda*x], [-4*Lambda*x, 2*Lambda]])
HH = lambda x,y: np.linalg.inv(ddf(x,y))

level_lines(f, .8, 1.2, 0.8, 1.2, np.linspace(0, 30, 80))
draw_vector_field(df, .8, 1.2, 0.8, 1.2, 15)
plt.plot(1,1,'or')
plt.axis('equal')
plt.show()

----------------------------------------------------------------------------------------------------------------

__Question 17.__ Implémentez la méthode de Newton et appliquez-la à la fonction ci-dessus avec $c=0.1$, $\beta=0.75$ et $x^0=(0,0)$. Représentez les itérations sur un graphique et tracez $\ \log(f_\Lambda(x^k))\ $ en fonction de $k$. Commentez les résultats.

_Indication:_ Testez d'abord l'algorithme sur la fonction quadratique ci-dessous.

In [ ]:
f = lambda x,y : ( x - 1)**2 + 2*(y - 1)**2
df = lambda x,y : np.array([2*(x - 1) , 4*(y - 1)])
#ddf = lambda x,y : np.array([[2  , 0], [0, 2]])
HH = lambda x,y : np.array([[.5, 0], [0, .25]])


In [ ]:
f = lambda x,y : ( x - 1)**2 + Lambda*(y - x**2)**2
df = lambda x,y : np.array([2*(x-1)+4*Lambda*x*(x**2-y),2*Lambda*(y-x**2)])
ddf = lambda x,y : np.array([[2+4*Lambda*(3*x**2-y), -4*Lambda*x], [-4*Lambda*x, 2*Lambda]])
HH = lambda x,y: np.linalg.inv(ddf(x,y))

In [ ]:
## Parameters
c, beta = .1, .9
epsilon = 1e-8
itermax = 200
iter_ls_max = 40

## initialization 
iter = 0
x, y, alpha = 0, 0, 1. 
fz = f(x,y)
W, F =[np.array([x, y])], [fz]
flag = 'OK'

## Optmization loop
while (iter < itermax):
    g = df(x,y)
    hh = HH(x,y)
    dx,dy = -np.dot(hh, g)
    d = np.hypot(dx, dy)
    if d < epsilon or flag == 'Not OK':
        break
   
    new_fz = f(x + alpha*dx, y + alpha*dy) 
    iter_ls = 0
    dd = d**2
    while (new_fz - fz + c*alpha*dd >=0):
        alpha *= beta
        new_fz = f(x + alpha*dx, y + alpha*dy)
        iter_ls += 1
        if (iter_ls>=iter_ls_max):
            flag = 'Not OK'
            break
    #print("d = " + str(d) + ", f(z) - 1 =" + str(fz-1) + ", t= " +str(t))
    x, y, fz = x + alpha*dx, y + alpha*dy, new_fz
    W.append(np.array([x, y]))
    F.append(fz)
    alpha /= beta
    iter += 1

print('flag = '+flag + ', n_iter = ' + str(iter))    
W = np.array(W)
F = np.array(F)

print(f"On trouve comme valeur de (x,y) = ({np.round(x,3)},{np.round(y,3)})")

# plot the results 
plt.figure()
plt.plot(W[:,0],W[:,1],'.',linestyle='-')
level_lines(f, 0, 2, 0, 2, np.linspace(1, 3, 10))
draw_vector_field(df, 0 , 2, 0, 2, 10)
plt.axis('equal')
plt.show()

# plot of the values of f along the iterations.

plt.figure() # NEW
plt.plot(np.arange(0, iter+1),np.log(F),'.',linestyle='-') # NEW
plt.ylabel('log(f(x,y))') # NEW
plt.xlabel('nb_iterations') # NEW
plt.title('Evolution of log(f) along the iterations') # NEW
plt.show() # NEW

<u>Interprétation</u> : L'algorithme converge vers une solution : (x,y) = (1.0,1.0). On remarque que cette convergence se traduit bien par une diminution de f(x,y) à chaque itération.

<a id='BFGS'></a>
<h2> Une méthode de quasi-Newton (BFGS)</h2>
<hr>

Lorsque le nombre de paramètres est important comme il est habituel en Machine Learning, le calcul des matrices hessiennes $D^2f(x^k)$ et la résolution des systèmes linéaires $D^2f(x^k) d^k=-g^k$ peuvent être trop coûteux. Cependant, il est souvent encore possible d'obtenir une convergence superlinéaire en remplaçant $[D^2f(x^k)]^{-1}$ par une approximation moins gourmande à calculer qu'on notera $H^k$. Il existe plusieurs algorithmes basés sur cette idée. Nous présentons l'une des plus populaires : la méthode BFGS du nom de leurs découvreurs (Broyden, Fletcher, Goldfarb et Shanno).

__Description de la méthode__ : Supposons qu'à l'étape $k$ nous ayons une approximation définie positive symétrique $H^k$ de $\left[D^2f(x^k)\right]^{-1}$. On note $B^k$ son inverse (qui est une approximation de $D^2f(x^k)$). Comme ci-dessus, nous définissons notre direction de descente $d^k$ comme le minimiseur de
$$
f(x^k) + d\cdot \nabla f(x^k) + \dfrac12 d^T B^k d.
$$
Cela conduit à la formule :
$$
d^k = -\left[B^k\right]^{-1} \nabla f(x^k) = - H^k g^k. 
$$

On cherche ensuite $\alpha_k$ satisfaisant (5) par la méthode de ``backtracking", toujours avec $\alpha=1$ et on pose
$$
x^{k+1} := x^k +\alpha_k d^k.
$$

Maintenant, nous avons besoin de calculer approximation $H^{k+1}$ de $\left[D^2f(x^{k+1})\right]^{-1}$. Pour cela, rappelons que nous voulons
$$
\tilde m_{k+1} (d):= f(x^{k+1}) + g^{k+1}\cdot d +\dfrac 12 d^T B^{k+1} d,
$$
soit une approximation de
$$
\overline m_{k+1}(d):= f(x^{k+1} + d).
$$
Nous avons déjà par construction,
$$
\tilde m_{k+1}(0)=\overline m_{k+1}(0)=f(x^{k+1})\qquad\text{et}\qquad \nabla \tilde m_{k +1}(0)=\nabla \overline m_{k+1}(0)=g(x^{k+1}).
$$
Nous appliquons la nouvelle condition
$$
\nabla m_{k+1}(-\tau_k d^k)=\nabla \overline m_{k+1}(-\tau_k d^k)=g^k.
$$

En notant $a^k:=g^{k+1}-g^k$ et $b^k:=\tau^kd^k=x^{k+1}-x^k$, cela équivaut à $B^{k+1}b^k=a^k$. En supposant que $B^{k+1}$ est inversible, cela équivaut à demander que $H^{k+1}$ soit solution de
$$
\tag{6}
Ha^k=b^k.
$$
Une condition nécessaire et suffisante pour que (6) admette une solution symétrique définie positive $H$ est :
$$
\tag{7}
\left<a^k;b^k\right> >0.
$$

Nous ne voulons pas perdre toute l'information déjà contenue dans $H^k$, donc, en supposant que (7) soit vraie, nous choisissons une solution de (6) aussi proche que possible de $H^k$. Un choix populaire consiste à définir :
$$
\tag{8}
H^{k+1} := \left(I-\rho_k b^k\otimes a^k\right) H^k \left(I-\rho_k a^k\otimes b^k\right) + \rho_k b^k\otimes b^k,\quad\text{ avec }\quad \rho_k:=\dfrac1{\left<a^k;b^k\right>}.
$$

__Question 18.__ Vérifiez que la formule (8) donne bien une solution à (6). Vérifiez que $H^{k+1}$ ainsi définie est une matrice symétrique définie positive 

<u>Réponse</u> : On doit vérifier que $H^{k+1} a^k = b^k$ On a

$H^{k+1} a^k = (H^k - \rho_k b^k\otimes a^k H^k)(I-\rho_k a^k\otimes b^k) a^k + \rho_k (b^k\otimes b^k) a^k$

$H^{k+1} a^k = H^k a^k + (\rho_k)^2 (b^k\otimes a^k) H^k (a^k\otimes b^k) a^k - \rho_k (b^k\otimes a^k) H^k a^k - \rho_k H^k (a^k\otimes b^k) a^k + \rho_k (b^k\otimes b^k) a^k$

$H^{k+1} a^k = b^k + (\rho_k)^2 (b^k\otimes a^k) H^k (a^k\otimes b^k) a^k - \rho_k (b^k\otimes a^k) b^k - \rho_k H^k (a^k\otimes b^k) a^k + \rho_k (b^k\otimes b^k) a^k$

Or, on sait qu'en dimension quelconque : $(a^k\otimes b^k) a^k = (b^k\otimes a^k) b^k = 0$ par orthogonalité du produit vectoriel. Donc :

$H^{k+1} a^k = b^k + \rho_k (b^k\otimes b^k) a^k$

Or, $(b^k\otimes b^k) a^k = b^k (b^k \otimes a^k) = 0$. Donc, on obtient finalement que :

$H^{k+1} a^k = b^k$

En conclusion, $H^{k+1}$ donne bien une solution à (6). De plus, comme $\left<a^k;b^k\right> >0$, alors d'après l'égalité précédente, $H^{k+1}$ est bien définie positive.

----------------------------------------------------------------------------------------------------------------

__Question 19.__ Implémentez la méthode BFGS et appliquez-la à la fonction ci-dessus avec $c = 0.1$, $\beta=0.75$ et $x^0=(0,0)$. Comme premier approximation de $D^2f(x^0)$ on prendra $H^0=I$.

Représentez les itérations sur un graphique et tracez $\ \log(f(x^k))\ $ en fonction de $k$. Observez et commentez.

__Question 20.__ Est-ce que $H^k$ converge vers $[D^2 f(x^*)]^{-1}$ ?

In [ ]:
## Solution

## Parameters
c, beta = .1, .75
epsilon = 1e-8
itermax = 200
iter_ls_max = 40

##
np.set_printoptions(precision=3)
np.set_printoptions(suppress="True")

## initialization 
iter = 0
x, y, alpha = 0.0, 0.0, 1. 
fz = f(x,y)
W, F =[np.array([x, y])], [fz]
flag = 'OK'
hh = np.identity(2)
h_error = []

## Optmization loop
while (iter < itermax):
    g = df(x,y)
    # hh = HH(x,y)
    hh = hh # NEW
    dx,dy = -np.dot(hh, g)
    d = np.hypot(dx, dy)
    if d < epsilon or flag == 'Not OK':
        break
   
    new_fz = f(x + alpha*dx, y + alpha*dy) 
    iter_ls = 0
    dd = d**2
    while (new_fz - fz + c*alpha*dd >=0):
        alpha *= beta
        new_fz = f(x + alpha*dx, y + alpha*dy)
        iter_ls += 1
        if (iter_ls>=iter_ls_max):
            flag = 'Not OK'
            break
    #print("d = " + str(d) + ", f(z) - 1 =" + str(fz-1) + ", t= " +str(t))
    b_k = np.array([[-x, -y]]) # NEW
    x, y, fz = x + alpha*dx, y + alpha*dy, new_fz
    a_k = np.array([df(x, y) - g]) # NEW
    b_k += np.array([[x, y]]) # NEW
    hh = np.dot(np.dot(np.identity(2) - (1/np.dot(a_k,b_k.T))*np.dot(b_k.T, a_k), hh), (np.identity(2) - (1/np.dot(a_k,b_k.T))*np.dot(a_k.T, b_k))) + (1/np.dot(a_k,b_k.T))*np.dot(b_k.T, b_k) # NEW
    W.append(np.array([x, y]))
    F.append(fz)
    alpha /= beta
    iter += 1
    if np.linalg.norm(hh-HH(x,y), 2) < 0.5:
        h_error.append(np.array([iter, np.linalg.norm(hh-HH(x,y), 2)])) # NEW

print('flag = '+flag + ', n_iter = ' + str(iter))    
W = np.array(W)
F = np.array(F)
h_error = np.array(h_error) # NEW

print(f"On trouve comme valeur de (x,y) = ({np.round(x,3)},{np.round(y,3)})") # NEW

# plot the results 
plt.figure()
plt.plot(W[:,0],W[:,1],'.',linestyle='-')
level_lines(f, 0, 2, 0, 2, np.linspace(1, 3, 10))
draw_vector_field(df, 0 , 2, 0, 2, 10)
plt.axis('equal')
plt.show()

# plot of the values of f along the iterations.

plt.figure() # NEW
plt.plot(np.arange(0, iter+1),np.log(F),'.',linestyle='-') # NEW
plt.ylabel('log(f(x,y))') # NEW
plt.xlabel('nb_iterations') # NEW
plt.title('Evolution of log(f) along the iterations') # NEW
plt.show() # NEW

# plot of the values of ||Happrox-Htrue|| along the iterations.
plt.figure() # NEW
plt.plot(h_error[:,0],h_error[:,1],'.',linestyle='-') # NEW
plt.ylabel('h_error') # NEW
plt.xlabel('nb_iterations') # NEW
plt.title('Evolution of ||Happrox-Htrue|| along the iterations') # NEW
plt.show() # NEW



<u>Interprétation</u> : On remarque que même avec l'approximation de la hessienne, on conserve une convergence vers la solution (x,y) = (1.0, 1.0). En particulier, il faut noter que $H_{approx}$ ne converge pas vers $H_{true}$...